In [44]:
import numpy as np
from PIL import Image
import os

# Custom sorting key function
def custom_sort_key(entry):
    return int(entry.split(".")[0][1:])

# Step 1: Preprocessing the training images
def preprocess_images(image_folder):
    images = []
    """
    entries = sorted(os.listdir(image_folder), key=custom_sort_key)
    for entry in entries:
        print(entry)
    """
    for filename in sorted(os.listdir(image_folder), key=custom_sort_key):
        img_path = os.path.join(image_folder, filename)
        img = Image.open(img_path).convert('L') #Convert to grayscale
        img = img.resize((100, 100)) #Resize for consistency
        img_data = np.array(img).flatten() # Flatten to 1D array
        images.append(img_data)
    return np.array(images)

# Step 2: Compute the mean face
def compute_mean_face(images):
    mean_face = np.mean(images, axis=0)
    return mean_face

#Step 3: Calculate the covariance matrix
def calculate_covariance_matrix(images):
    mean_face = compute_mean_face(images)
    centered_images = images - mean_face
    covariance_matrix = np.cov(centered_images, rowvar = False)
    return covariance_matrix

# Step 4: Compute eigenvectors and eigenvalues
def compute_eigenfaces(covariance_matrix):
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    sorted_indices = np.argsort(eigenvalues)[::-1] # Sort in descending order
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:,sorted_indices]
    return eigenvalues, eigenvectors

# Step 5: Select a subset of eigenvectors
def select_eigenvectors(eigenvalues, eigenvectors, num_components):
    selected_eigenvalues = eigenvalues[:num_components]
    selected_eigenvectors = eigenvectors[:, :num_components]
    return selected_eigenvalues, selected_eigenvectors

# Step 6: Project traning images onto eigenspace
def project_images(images, mean_face, eigenvectors):
    centered_images = images - mean_face
    projected_images = np.dot(centered_images, eigenvectors)
    return projected_images

In [45]:
# Main function
def main():
    # Step 1: Preprocess the training images
    image_folder = "train"
    images = preprocess_images(image_folder)
    
    # Step 2: Compute the mean face
    mean_face = compute_mean_face(images)
    
    # Step 3: Calculate the covariance matrix
    covariance_matrix = calculate_covariance_matrix(images)
    
    # Step 4: Compute eigenvectors and eigenvalues
    eigenvalues, eigenvectors = compute_eigenfaces(covariance_matrix)
    
    # Step 5: Select a subset of eigenvectors
    num_components = 10 # Adjust this based on desired dimensionality
    selected_eigenvalues, selected_eigenvectors = select_eigenvectors(eigenvalues, eigenvectors, num_components)
    
    #Step 6: Project training images onto eigenspace
    projected_images = project_images(images, mean_face, selected_eigenvectors)
    
    # Store the new low-dimensional representation of each training image
    np.savez("eigenfaces_representation.npz", projected_images=projected_images, mean_face=mean_face, eigenvectors=selected_eigenvectors )
    
if __name__ == "__main__":
    main()

In [46]:
import numpy as np
from PIL import Image
import os

# Load the low-dimensional representation of training images and mean face
data = np.load("eigenfaces_representation.npz")
projected_images = data["projected_images"]
mean_face = data["mean_face"]
eigenvectors = data["eigenvectors"]

# Function to project an image onto the low-dimensional space
def project_image(image_path, mean_face, eigenvectors):
    img = Image.open(image_path).convert('L').resize((100, 100))
    img_data = np.array(img).flatten()
    centered_img = img_data - mean_face
    projected_img = np.dot(centered_img, eigenvectors)
    return projected_img

# Function to find the closest match from the training set
def find_closest_match(projected_img, projected_images):
    distances = np.linalg.norm(projected_images - projected_img, axis = 1)
    closest_index = np.argmin(distances)
    return closest_index


print("Test recognition on the additional training images")
training_image_folder = "train"
for filename in os.listdir(training_image_folder):
    image_path = os.path.join(training_image_folder, filename)
    projected_img = project_image(image_path, mean_face, eigenvectors)
    closest_index = find_closest_match(projected_img, projected_images)
    print(f"Matched {filename} with testing image {closest_index + 1}")

# Test recognition on the additional test images
print("Test recognition on the additional test images")
test_image_folder = "test"
for filename in os.listdir(test_image_folder):
    image_path = os.path.join(test_image_folder, filename)
    projected_img = project_image(image_path, mean_face, eigenvectors)
    closest_index = find_closest_match(projected_img, projected_images)
    print(f"Matched {filename} with testing image {closest_index + 1}")

Test recognition on the additional training images
Matched s1.1.tif with testing image 1
Matched s10.1.tif with testing image 10
Matched s11.1.tif with testing image 11
Matched s12.1.tif with testing image 12
Matched s13.1.tif with testing image 13
Matched s14.1.tif with testing image 14
Matched s15.1.tif with testing image 15
Matched s16.1.tif with testing image 16
Matched s17.1.tif with testing image 17
Matched s18.1.tif with testing image 18
Matched s19.1.tif with testing image 19
Matched s2.1.tif with testing image 2
Matched s20.1.tif with testing image 20
Matched s21.1.tif with testing image 21
Matched s22.1.tif with testing image 22
Matched s23.1.tif with testing image 23
Matched s24.1.tif with testing image 24
Matched s25.1.tif with testing image 25
Matched s26.1.tif with testing image 26
Matched s27.1.tif with testing image 27
Matched s28.1.tif with testing image 28
Matched s29.1.tif with testing image 29
Matched s3.1.tif with testing image 3
Matched s30.1.tif with testing imag